# Projet Optimisation de portefeuille - EDA

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import json
import sys

project_root = os.path.dirname(os.path.abspath(''))
raw_data = os.path.join(project_root, 'data', 'raw')
processed_data = os.path.join(project_root, 'data', 'processed')

if project_root not in sys.path:
    sys.path.append(project_root)

from src.data_prep import all_sectors, all_sectors_with_sector

### Création et chargement des données

In [2]:
all_sectors()
all_sectors_with_sector()

df = all_sectors()
df_with_sector = all_sectors_with_sector()

df.head()

Fichier all_sectors.csv sauvegardé
Nombre total de lignes : 2516
Fichier all_sectors_with_sector.csv sauvegardé
Nombre total de lignes : 490056
Fichier all_sectors.csv sauvegardé
Nombre total de lignes : 2516
Fichier all_sectors_with_sector.csv sauvegardé
Nombre total de lignes : 490056


,GOOGL,META,DIS,NFLX,VZ,T,TMUS,CMCSA,CHTR,EA,...,WELL,EQIX,NEE,DUK,SO,D,AEP,EXC,SRE,XEL
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,26.296135,77.969330,85.943504,4.984857,26.573196,11.657849,26.374689,22.256111,166.429993,45.571419,...,50.148003,175.042145,20.032852,53.362507,30.940659,49.050907,41.371307,18.123081,40.092171,25.736334
2015-01-05,25.795088,76.717064,84.687592,4.731143,26.352507,11.547699,26.017231,21.802223,160.500000,45.279728,...,50.767151,173.242645,19.820673,52.594292,30.809050,48.455883,40.762409,17.606936,39.358940,25.444193
2015-01-06,25.158480,75.683441,84.238403,4.650143,26.618471,11.564913,25.959267,21.517820,156.309998,44.686634,...,51.490532,169.466141,19.897659,53.457737,31.178810,48.209019,40.985680,17.471869,38.990517,25.579571
2015-01-07,25.084490,75.683441,85.100121,4.674286,26.446693,11.632341,27.292492,21.498335,156.789993,45.513073,...,51.966255,170.022141,20.079798,53.629143,31.523483,48.405258,41.567516,17.495985,39.620033,25.800451
2015-01-08,25.171888,77.700996,85.980171,4.778000,27.013542,11.748068,28.036390,21.965860,162.940002,46.796505,...,51.751202,172.053329,20.201839,53.845024,31.548576,48.974937,41.926086,17.631058,39.974129,26.099705


#### Calcul des rendements journaliers
Formule : (Prix_jour_J - Prix_jour_J-1) / Prix_jour_J-1

In [3]:
returns = df.pct_change()
returns.drop(returns.index[0], inplace=True)

missing_values = returns.isnull().sum()
non_zero_missing = missing_values[missing_values > 0]
if not non_zero_missing.empty:
    print(f"Nombre de valeurs manquantes :\n{non_zero_missing}")

Nombre de valeurs manquantes :
KHC      126
OTIS    1311
SNOW    1436
SHOP      95
CRWD    1117
ZS       806
MDB      705
dtype: int64


In [4]:
# On nettoie les colonnes qui contiennent des NaN, cela va retirer KHC, OTIS, SNOW, etc. mais garder tout l'historique 2015-2024
# on fait ça car on ne doit avoir aucune valeur manquante pour la suite
returns_clean = returns.dropna(axis=1)

print(f"Avant nettoyage : {returns.shape[1]} actifs")
print(f"Après nettoyage : {returns_clean.shape[1]} actifs")
print(f"Valeurs manquantes restantes : {returns_clean.isnull().sum().sum()}")

Avant nettoyage : 197 actifs
Après nettoyage : 190 actifs
Valeurs manquantes restantes : 0


Export

In [5]:
returns_clean.to_csv(os.path.join(processed_data, 'daily_returns.csv'))